In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from transformers.models.bert.configuration_bert import BertConfig
import numpy as np
import os
import logging
import json


with open('config.json', 'r') as f:
    config = json.load(f)

path = config['working_dir']

output_dir = os.path.join(path,'output_bert_no_dim_new_new3') # Directory di output
print('output_dir: ',output_dir)
os.makedirs(output_dir, exist_ok=True)  # Crea la directory di output se non esiste

# Configurazione del logger
logging.basicConfig(filename=os.path.join(output_dir, 'txt_embedding_generation.log'), 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [ ]:
file_path = config['nodes_file_path']  # Percorso del file TSV di input
print('nodes_file_path:',file_path)

# Carica solo le colonne necessarie dal file TSV
logger.info("Caricamento del file TSV...")
df = pd.read_csv(file_path, sep="\t", usecols=["name", "type", "Description", "Sequence"])
logger.info(f"File TSV caricato. Numero di righe: {len(df)}")

# Carica i modelli e i tokenizer
logger.info("Caricamento dei modelli e tokenizer...")
biobert_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2", trust_remote_code=True)
biobert_model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.2", trust_remote_code=True)

# Crea la colonna 'embedding' se non esiste
df["embedding"] = None

In [ ]:

# Funzione per ottenere embedding da BioBERT
def get_text_embedding(text):
    if pd.isna(text):  # Se il testo è NaN, restituiamo None
        return None
    try:
        inputs = biobert_tokenizer(text, return_tensors='pt')['input_ids']
        hidden_states = biobert_model(inputs)[0]  # [1, sequence_length, 768]

        embedding_mean = torch.mean(hidden_states[0], dim=0)  # Mean pooling

        return embedding_mean.detach().cpu().numpy().tolist()  # Converti in lista
    except Exception as e:
        logger.error(f"Errore durante il calcolo dell'embedding per il testo: len {len(text)}  {text}. \nErrore: {e}")
        return None

# Funzione per salvare i dati in batch
def save_batch(df, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    
    # Se il file esiste già, caricalo e aggiungi i nuovi dati
    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path, sep="\t")
        df = pd.concat([existing_df, df], ignore_index=True)
    
    # Salva solo le colonne 'name' e 'embedding'
    df[["name", 'type', "embedding"]].to_csv(file_path, sep="\t", index=False)
    os.sync()  # Sincronizza i buffer del sistema operativo
    logger.info(f"Batch salvato in {file_path}")

# Funzione per caricare gli indici già processati
def load_processed_indices(file_path):
    if os.path.exists(file_path):
        return set(pd.read_csv(file_path, sep="\t").index.tolist())
    return set()

In [ ]:
# Verifica se tutti gli embedding di testo sono già stati calcolati
text_output_path = os.path.join(output_dir, "text_embeddings.tsv")
text_output_filled_path = os.path.join(output_dir, "text_embeddings_filled.tsv")
processed_indices_path = os.path.join(output_dir, "processed_text_indices.txt")

In [ ]:
# ---- 1. Calcola gli embedding per BioBERT (Phenotype, Disease, Genomic feature) ----
logger.info("Inizio calcolo embedding per BioBERT...")
df_text = df[df["type"].isin(["Phenotype", "Disease", "Genomic feature"])].copy()
logger.info(f"Text shape {df_text.shape}")
df_text

processed_indices = load_processed_indices(processed_indices_path)

if os.path.exists(text_output_path):
    existing_df = pd.read_csv(text_output_path, sep="\t")
    if len(existing_df) == len(df_text):  # Tutti gli embedding sono già stati calcolati
        logger.info("Tutti gli embedding di testo sono già stati calcolati. Saltando il calcolo.")
        df_text = existing_df
    else:
        # Calcola gli embedding mancanti
        logger.info("Calcolo degli embedding di testo mancanti...")
        for i, row in df_text.iterrows():
            if i in processed_indices:  # Salta gli embedding già calcolati
                continue
            print(row["Description"])
            embedding = get_text_embedding(row["Description"])
            df_text.at[i, "embedding"] = embedding  # Assegna l'embedding alla riga
            
            # Salva ogni 10 embedding
            if (i + 1) % 10 == 0:
                save_batch(df_text.iloc[i-9:i+1], text_output_path)
                with open(processed_indices_path, "a") as f:
                    f.write("\n".join(map(str, range(i-9, i+1))) + "\n")
                logger.info(f"Salvati {i+1} embedding di testo")

        # Salva gli ultimi embedding rimanenti (se ce ne sono)
        remaining_indices = range(len(df_text) - (len(df_text) % 10), len(df_text))
        if len(remaining_indices) > 0:
            save_batch(df_text.iloc[remaining_indices], text_output_path)
            with open(processed_indices_path, "a") as f:
                f.write("\n".join(map(str, remaining_indices)) + "\n")
            logger.info(f"Salvati gli ultimi {len(remaining_indices)} embedding di testo")
else:
    # Calcola tutti gli embedding di testo
    logger.info("Calcolo di tutti gli embedding di testo...")
    for i, row in df_text.iterrows():
        embedding = get_text_embedding(row["Description"])
        df_text.at[i, "embedding"] = embedding  # Assegna l'embedding alla riga
        
        # Salva ogni 10 embedding
        if (i + 1) % 10 == 0:
            save_batch(df_text.iloc[i-9:i+1], text_output_path)
            with open(processed_indices_path, "a") as f:
                f.write("\n".join(map(str, range(i-9, i+1))) + "\n")
            logger.info(f"Salvati {i+1} embedding di testo")

    # Salva gli ultimi embedding rimanenti (se ce ne sono)
    remaining_indices = range(len(df_text) - (len(df_text) % 10), len(df_text))
    if len(remaining_indices) > 0:
        save_batch(df_text.iloc[remaining_indices], text_output_path)
        with open(processed_indices_path, "a") as f:
            f.write("\n".join(map(str, remaining_indices)) + "\n")
        logger.info(f"Salvati gli ultimi {len(remaining_indices)} embedding di testo")


In [37]:

# ---- 3. Sostituisci i valori nulli con la media degli embedding del tipo corrispondente ----
logger.info("Sostituzione dei valori nulli con la media degli embedding...")


def replace_null_embeddings_with_type_mean(df):
    """
    Sostituisce gli embedding nulli con la media degli embedding non nulli per tipo.
    
    Args:
        df (pd.DataFrame): DataFrame contenente le colonne 'name', 'type', 'len_seq', 'embedding'
        
    Returns:
        pd.DataFrame: DataFrame con gli embedding nulli sostituiti
    """
    # Converti la colonna embedding da stringa a lista di float (se necessario)
    if isinstance(df['embedding'].iloc[0], str):
        df['embedding'] = df['embedding'].apply(lambda x: eval(x) if pd.notna(x) else np.nan)
    
    # Converti le liste in array numpy
    df['embedding'] = df['embedding'].apply(lambda x: np.array(x) if isinstance(x, list) else x)
    
    # Filtra i record con embedding non nulli
    non_null_mask = df['embedding'].apply(lambda x: x is not np.nan if isinstance(x, np.ndarray) else pd.notna(x))
    non_null_embeddings = df[non_null_mask]
    
    # Calcola la media degli embedding per ogni tipo
    type_mean_embeddings = non_null_embeddings.groupby('type')['embedding'].apply(
        lambda x: np.mean(np.stack(x.values), axis=0)
    ).to_dict()
    
    # Sostituisci gli embedding nulli con la media del loro tipo
    def fill_na_embedding(row):
        if isinstance(row['embedding'], np.ndarray):
            return row['embedding']
        elif pd.isna(row['embedding']):
            return type_mean_embeddings.get(row['type'], np.nan)
        return row['embedding']
    
    df['embedding'] = df.apply(fill_na_embedding, axis=1)

    def to_list(embedding):
        if isinstance(embedding, str):
            embedding = np.array(eval(embedding))
        return embedding.tolist()
    
    # Converte gli array numpy in liste per una corretta scrittura su file come string
    df['embedding'] = df['embedding'].apply(to_list)
    
    return df

df_text = pd.read_csv(text_output_path, sep="\t")

df_text_filled = replace_null_embeddings_with_type_mean(df_text)
print(df_text_filled[df_text_filled['embedding'].isna()])  # Dovrebbe essere vuoto a meno che non ci siano tipi senza esempi validi

# ---- 4. Salva i risultati finali ----
if os.path.exists(text_output_filled_path):
    logger.info("Esiste già un file. Saltando il calcolo.")
    pass 
else:
    try:
        logger.info("Salvataggio dei file finali...")
        save_batch(df_text_filled, text_output_filled_path)
        logger.info(f"File finali salvati: {text_output_filled_path} ")
    except Exception as e:
        logger.error(f"Errore durante il salvataggio dei file finali: {e}")

Empty DataFrame
Columns: [name, type, embedding]
Index: []


In [38]:
(0.11967191100120544+0.18104693293571472+0.16617423295974731+0.07239675521850586)/4
# 0.13482245802879333

0.13482245802879333